In [1]:
#The csv file containing the music_info.csv and created cluster labels imported as a data frame through Pandas
import pandas as pd

In [2]:
threeclusters_df = pd.read_csv("3clusters_musicinfo.csv")
threeclusters_df.head(5)

,Unnamed: 0,name,artist,danceability,energy,key,mode,valence,tempo,Cluster
0,0,Mr. Brightside,The Killers,0.355,0.918,1,1,0.240,0.619996,1
1,1,Wonderwall,Oasis,0.409,0.892,2,1,0.651,0.730137,1
2,2,Come as You Are,Nirvana,0.508,0.826,4,0,0.543,0.502363,1
3,3,Take Me Out,Franz Ferdinand,0.279,0.664,9,1,0.490,0.437682,2
4,4,Creep,Radiohead,0.515,0.430,7,1,0.104,0.384441,0


In [3]:
threeclusters_df['X_vector'] = [[threeclusters_df.danceability.iloc[i],threeclusters_df.energy.iloc[i],threeclusters_df.key.iloc[i],threeclusters_df['mode'].iloc[i],threeclusters_df.valence.iloc[i],threeclusters_df.tempo.iloc[i]] for i in range(0, len(threeclusters_df))]
threeclusters_df.head(5)

,Unnamed: 0,name,artist,danceability,energy,key,mode,valence,tempo,Cluster,X_vector
0,0,Mr. Brightside,The Killers,0.355,0.918,1,1,0.240,0.619996,1,"[0.355, 0.918, 1, 1, 0.24, 0.6199962326545135]"
1,1,Wonderwall,Oasis,0.409,0.892,2,1,0.651,0.730137,1,"[0.409, 0.892, 2, 1, 0.651, 0.7301366709223718]"
2,2,Come as You Are,Nirvana,0.508,0.826,4,0,0.543,0.502363,1,"[0.508, 0.826, 4, 0, 0.543, 0.5023629628079282]"
3,3,Take Me Out,Franz Ferdinand,0.279,0.664,9,1,0.490,0.437682,2,"[0.279, 0.664, 9, 1, 0.49, 0.4376818267439671]"
4,4,Creep,Radiohead,0.515,0.430,7,1,0.104,0.384441,0,"[0.515, 0.43, 7, 1, 0.104, 0.3844408631407104]"


In [4]:
#import the necessary library, modules, functions, settings 
import sklearn
import random
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [5]:
#Define the input and target variable
X = threeclusters_df['X_vector']
y = threeclusters_df['Cluster']

In [6]:
#We split the data into a train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [7]:
#convert to lists of values 
X_train_vals = X_train.tolist()
X_test_vals = X_test.tolist()
y_train_vals = y_train.tolist()
y_test_vals = y_test.tolist()

In [8]:
#scale the data
scaler = StandardScaler()
scaler.fit(X_train_vals)
X_train_vals = scaler.transform(X_train_vals)
X_test_vals = scaler.transform(X_test_vals)  

In [9]:
#define functions that create batches and then shuffle those batches
batch_size = 500
batch_number = int(np.ceil(len(X_train) / float(batch_size)))
shuffle_number = range(len(X_train))

def create_batches(iterable_X, iterable_y):
    X_batches = []
    y_batches = []
    for i in range(batch_number):
        X_batches.append(iterable_X[i*batch_size:(i+1)*batch_size])
        y_batches.append(iterable_y[i*batch_size:(i+1)*batch_size])
    return X_batches, y_batches
    

def shuffle_batch(iterable_X, iterable_y):
    indices = [i for i in shuffle_number]
    random.shuffle(indices)
    shuffledX = [iterable_X[i] for i in indices]
    shuffledy = [iterable_y[i] for i in indices]
    return shuffledX, shuffledy

In [10]:
#initialize an instance of SGD
SGDC = SGDClassifier(loss="hinge", penalty="l2", alpha=0.0001, max_iter=3000, tol=None, shuffle=True, verbose=0, learning_rate='adaptive', eta0=0.01, early_stopping=False, random_state=123)

In [11]:
#create the training batches
X_train_batches, y_train_batches = create_batches(X_train_vals,y_train_vals)

In [12]:
#iterative learning using the batches and evaluating each partial fit
for epoch in range(10): 
    for chunk_X, chunk_y in zip(X_train_batches, y_train_batches):
        shuffle_number = range(len(chunk_X))
        chunk_X, chunk_y = shuffle_batch(chunk_X, chunk_y)
        SGDC.partial_fit(chunk_X, chunk_y, classes=[0, 1, 2])
        y_predicted = SGDC.predict(X_test_vals)
        print(f'accuracy_score: {accuracy_score(y_test_vals, y_predicted)}',f"f1 score: {f1_score(y_test_vals, y_predicted, average='weighted')}",f"precision score: {precision_score(y_test_vals, y_predicted, average='weighted')}",f"recall score: {recall_score(y_test_vals, y_predicted, average='weighted')}")   

/opt/anaconda3/envs/erdos_may_2024_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


accuracy_score: 0.6895531222255105 f1 score: 0.5669013769949234 precision score: 0.48507476624814977 recall score: 0.6895531222255105
accuracy_score: 0.7011936470356122 f1 score: 0.5914793470917835 precision score: 0.7970267394564537 recall score: 0.7011936470356122
accuracy_score: 0.720232810496202 f1 score: 0.6317344250390795 precision score: 0.8074654829293504 recall score: 0.720232810496202
accuracy_score: 0.7596922166321397 f1 score: 0.7027480146415324 precision score: 0.8246435770362625 recall score: 0.7596922166321397
accuracy_score: 0.7918516326329289 f1 score: 0.753616210219815 precision score: 0.8417583717307802 recall score: 0.7918516326329289
accuracy_score: 0.7668935582519483 f1 score: 0.7158178588736145 precision score: 0.8313855951016882 recall score: 0.7668935582519483
accuracy_score: 0.8034921574430305 f1 score: 0.7710290836820292 precision score: 0.8492344860294992 recall score: 0.8034921574430305
accuracy_score: 0.8027029693203117 f1 score: 0.7695448657271227 precisi